In [1]:
#Purpose: explore my network
import uuid
import networkx as nx
import numpy as np

In [3]:
class Wire:
    def __init__(self, router1, router2):
        # connection between two routers
        self.router1 = router1
        self.router2 = router2
        # need a tuple to jump on graph
        self.jump = (router1.id, router2.id)
        # list of packets on wire
        # acting as input buffer
        self.packets = []

    def __eq__(self, other):
        if not isinstance(self, other):
            return False

        return (self.router1 is other.router1) and (self.router2 is other.router2)
    
    def __repr__(self):
        return f"{self.router1.id},{self.router2.id}\n"

    def find_packet(self, packet):
        for p in self.packets:
            if p is packet:
                return packet

        return None

    def insert_packet(self, packet):
        self.packets.append(packet)

    def remove_packet(self, packet, dst):
        p = self.find_packet(packet)

        if dst == self.router1.id:
            return self.hop(p, self.router1)
        elif dst == self.router2.id:
            return self.hop(p, self.router2)

    def hop(self, packet, dst):
        # if we successfully hopped to router, remove packet
        if packet and dst.insert_packet(packet):
            self.packets.remove(packet)
            return dst

        # operation failure, dst router congested
        return None


In [4]:
class Router:
    def __init__(self, id, kind, network, buffer_size):
        # node on graph
        self.id = id
        self.network = network
        # avoiding using type, better name?
        self.kind = kind

        # edges connected to this node
        # list of wires connected to routers
        self.connections = []

        self.actions = []

        # depending on type, set buffer size
        if kind == 'T':
            self.buffer_size = 100  # change this to hold more packets
        elif kind == 'M':
            self.buffer_size = 25  # change this to hold more packets
        elif kind == 'C' or kind == 'CP':
            self.buffer_size = buffer_size

        self.buffer = []  # Queue

    def __eq__(self, other):
        if not isinstance(self, other):
            return False

        return self.id == other.id

    def __repr__(self):
        return f"id: {self.id}, type: {self.kind}\n connections: {self.connections}\n"

    # for visualization we need to know
    # if the router is full, active, and inactive
    # full = red; active = yellow; inactive = grey
    def is_full(self):
        return len(self.buffer) >= self.buffer_size

    def is_active(self):
        return len(self.buffer) != 0

    # can only add and find connections
    def has_connection(self, dst):
        for connection in self.connections:
            node1, node2 = connection.jump
            if dst == node1 or dst == node2:
                return connection

        return None

    def build_connections(self, wires):
        for wire in wires:
            if self is wire.router1 or self is wire.router2:
                self.connections.append(wire)

    def build_actions(self):
        for connection in self.connections:
            r1, r2 = connection.jump
            if r1 != self.id:
                self.actions.append(r1)
            elif r2 != self.id:
                self.actions.append(r2)

    # no need for queue, just find and remove
    # todo: if this is the packet dst router, call network to remove packet from transmission

    def insert_packet(self, packet):
        if self is packet.dst:
            return True

        if not self.is_full():
            # enqueue packet
            self.buffer.append(packet)
            return True

        return False

    # push to wire
    def remove_packet(self, packet, dst):
        # if there is a connection to the destination
        # and the buffer is not empty
        # and the packet exists
        # move packet to wire
        wire = self.has_connection(dst)
        packet_index = self.find_packet(packet)
        if wire and self.buffer and packet_index != -1:
            # dequeue packet
            self.buffer.remove(packet)
            wire.insert_packet(packet)
            return wire

    def find_packet(self, packet):
        for index, p in enumerate(self.buffer):
            if p is packet:
                return index
        return -1

    # cant generate packets here, must generate from network and insert to routers
    # centralized controller controls packet path


In [5]:
class Packet:
    def __init__(self, src, dst, graph, path=[]):
        self.id = str(uuid.uuid4())
        self.curr = src
        self.curr_router = src  # we need to know src router for each hop on wire
        self.next_router = None  # updated on path update
        self.src = src
        self.dst = dst
        self.graph = graph  # change this to network on vscode
        self.path = self.update_path()

# possibly return possible actions in current router
# possible actions = where we can hop

    # add an equal method to find
    def __eq__(self, other):
        if not isinstance(self, other):
            return False

        return self.id == other.id

    def __repr__(self):
        return f"id: {self.id},\n curr: {self.curr_router}, dst: {self.dst}"

    def on_wire(self):
        return isinstance(self.curr, Wire)

    def on_router(self):
        return isinstance(self.curr, Router)

    # this is the input buffer for the next router
    def push_to_wire(self):
        if isinstance(self.curr, Router):
            self.curr = self.curr.remove_packet(self, self.next_router)

    def push_to_router(self):
        if isinstance(self.curr, Wire):
            router = self.curr.remove_packet(self, self.next_router)
            if router:
                self.curr_router = router
                self.curr = router
                self.update_path()
            # otherwise we stay on wire

    def complete(self):
        return self.curr is self.dst

    # build a more sophisticated path finding function, for now, shortest path
    def update_path(self):
        self.path = nx.shortest_path(
            self.graph, self.curr_router.id, self.dst.id)
        if len(self.path) >= 2:
            self.next_router = self.path[1]


In [6]:
class Network:
    def __init__(self, router_num=25, buffer_size=10):
        # initialize networkx backend
        self.network = self.init_network(router_num)

        # initialize network variables

        # all the routers and wires
        self.routers = []
        self.wires = []

        # the routers that can generate new packets
        self.customer_routers = []
        self.customer_buffer_size = buffer_size

        # total amount of packets
        self.packets = []

        self.all_router_data = np.array(self.network.nodes.data())
        self.all_connections = np.array(self.network.edges())

        # build network
        self.build_network()

    def shortest_path_step(self):
        for packet in self.packets:
            # it doesnt work without this, why?
            packet = self.packets[0]
            if not packet.complete():
                packet.push_to_wire()
                packet.push_to_router()
            else:
                print(packet)
                self.packets.remove(packet)

    def generate_packets(self, amount=1):
        if not amount > self.customer_buffer_size:
            for router in self.customer_routers:
                src = self.routers[router.id]
                for _ in range(amount):
                    dst = np.random.choice(self.customer_routers)
                    # dst cannot be the same src
                    # you cannot send packets to yourself
                    while src is dst:
                        dst = np.random.choice(self.customer_routers)
                    else:
                        p = Packet(src, dst, self.network)
                        self.routers[src.id].insert_packet(p)
                        self.packets.append(p)
        else:
            raise Exception(
                f"cannot generate more packets than buffer size: {self.customer_buffer_size}")

    # low level initialization details

    def init_network(self, n):
        graph = nx.random_internet_as_graph(n)
        nx.set_node_attributes(graph, {})
        return graph

    def build_network(self):
        self.build_routers(self.customer_buffer_size)
        self.build_wires()
        self.connect_routers()

    def connect_routers(self):
        for router in self.routers:
            router.build_connections(self.wires)
            router.build_actions()

    def build_routers(self, buffer_size):
        # extract data from nodes and edges and map
        for node in self.all_router_data:
            id = node[0]
            kind = node[1]['type']
            r = Router(id=id, kind=kind, network=self.network,
                       buffer_size=buffer_size)
            self.routers.append(r)

            if kind == 'C' or kind == 'CP':
                self.customer_routers.append(r)

    def build_wires(self):
        for connection in self.all_connections:
            router_id_1 = connection[0]
            router_id_2 = connection[1]
            wire = Wire(self.routers[router_id_1], self.routers[router_id_2])
            self.wires.append(wire)

In [7]:
N = Network()

N.generate_packets(9)

while N.packets:
    # prints packets when they arrive at destination
    N.shortest_path_step()

id: 802d7b0e-3a35-4178-812e-b6a13ef2d5a1,
 curr: id: 24, type: C
 connections: [5,24
]
, dst: id: 24, type: C
 connections: [5,24
]

id: 4a639960-f82a-455b-849f-64279c78cc39,
 curr: id: 22, type: C
 connections: [3,22
]
, dst: id: 22, type: C
 connections: [3,22
]

id: c318d2e9-ad56-450e-891f-1c43a80bc662,
 curr: id: 12, type: C
 connections: [3,12
]
, dst: id: 12, type: C
 connections: [3,12
]

id: 6770e3b8-1b9a-4a17-b280-175cbcd749be,
 curr: id: 18, type: C
 connections: [8,18
]
, dst: id: 18, type: C
 connections: [8,18
]

id: 9894c7e7-42bf-45f5-bf8d-90d2b280fcf1,
 curr: id: 24, type: C
 connections: [5,24
]
, dst: id: 24, type: C
 connections: [5,24
]

id: 55c6f6ca-3a9c-4441-9c20-bd8f27b3de3d,
 curr: id: 18, type: C
 connections: [8,18
]
, dst: id: 18, type: C
 connections: [8,18
]

id: 4437cd61-92db-4b47-9885-927bf239d550,
 curr: id: 23, type: C
 connections: [7,23
]
, dst: id: 23, type: C
 connections: [7,23
]

id: 6abf8bfe-e4b9-4f43-83fa-7889b28da53b,
 curr: id: 18, type: C
 con

Streaming output truncated to the last 5000 lines.

id: 713591b6-3ae6-48be-9918-8927e5ed2eeb,
 curr: id: 18, type: C
 connections: [7,18
]
, dst: id: 18, type: C
 connections: [7,18
]

id: 332f4ffd-925d-4314-9313-dcff596a31f8,
 curr: id: 8, type: M
 connections: [0,8
, 4,8
, 6,8
, 7,8
, 8,11
, 8,23
, 8,24
]
, dst: id: 16, type: C
 connections: [7,16
]

id: 332f4ffd-925d-4314-9313-dcff596a31f8,
 curr: id: 7, type: M
 connections: [1,7
, 2,7
, 5,7
, 6,7
, 7,9
, 7,14
, 7,16
, 7,18
, 7,19
, 7,8
]
, dst: id: 16, type: C
 connections: [7,16
]

id: 332f4ffd-925d-4314-9313-dcff596a31f8,
 curr: id: 16, type: C
 connections: [7,16
]
, dst: id: 16, type: C
 connections: [7,16
]

id: d7c5c6bb-1d0e-4c83-90cc-1c4d21e39ef8,
 curr: id: 8, type: M
 connections: [0,8
, 4,8
, 6,8
, 7,8
, 8,11
, 8,23
, 8,24
]
, dst: id: 20, type: C
 connections: [4,20
]

id: d7c5c6bb-1d0e-4c83-90cc-1c4d21e39ef8,
 curr: id: 4, type: T
 connections: [0,4
, 1,4
, 2,4
, 3,4
, 4,5
, 4,8
, 4,20
]
, dst: id: 20, type: C
 connect